<a href="https://colab.research.google.com/github/aShahawy/GP-Test/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data Prep**

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1badu11NqxGf6qM3PTTooQDJvQbejgbTv/view
!gdown --fuzzy https://drive.google.com/file/d/1WFl3P9CF23RGhCskLrgm15vEEumvsRWv/view
!gdown --fuzzy https://drive.google.com/file/d/1X1EFCyralNN2Bg3LhelL_lShrSrmTitW/view

In [ ]:
!unzip "CelebAMask-HQ.zip"

In [ ]:
!unzip "sketch.zip"

In [ ]:
!unzip "text.zip"

In [ ]:
# !cp /content/sketch/900.jpg /content/174.jpg

In [ ]:
import matplotlib.pyplot as plt
import shutil
for i in range(50,100):
  # shutil.rmtree("/content/"+str(i)+"_s.jpg")
  # shutil.rmtree("/content/"+str(i)+"_o.jpg")
# for i in range(50,100):
#   shutil.copy ("/content/sketch/"+str(i)+".jpg", "/content/"+str(i)+"_s.jpg")
#   shutil.copy ("/content/CelebAMask-HQ/CelebA-HQ-img/"+str(i)+".jpg", "/content/"+str(i)+"_o.jpg")

In [24]:
!curl -LO https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    93    0    93    0     0    600      0 --:--:-- --:--:-- --:--:--   600
100  281M  100  281M    0     0  46.5M      0  0:00:06  0:00:06 --:--:-- 48.1M


# Using *StyleGAN*

In [ ]:
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [ ]:
!pip install numpy==1.20 click==8.0 pillow==8.3.1 scipy==1.7.1 \
requests==2.26.0 tqdm==4.62.2 ninja==1.10.2 matplotlib==3.4.2 imageio==2.9.0 imgui==1.3.0 \
glfw==2.2.0 pyopengl==3.1.5 imageio-ffmpeg==0.4.3 pyspng

In [22]:
!git clone https://github.com/NVlabs/stylegan3
!cp /content/stylegan3/dnnlib /content/dnnlib -r
!cp /content/stylegan3/torch_utils /content/torch_utils -r
!touch /content/stylegan3/no.py
# !git clone https://github.com/NVlabs/stylegan3
!git clone https://github.com/openai/CLIP
!pip install -e ./CLIP
!pip install einops ninja


Cloning into 'stylegan3'...
remote: Enumerating objects: 193, done.
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (193/193), 4.18 MiB | 30.11 MiB/s, done.
Resolving deltas: 100% (86/86), done.
Cloning into 'CLIP'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 195 (delta 12), reused 22 (delta 9), pack-reused 168
Receiving objects: 100% (195/195), 8.91 MiB | 22.52 MiB/s, done.
Resolving deltas: 100% (94/94), done.
Obtaining file:///content/CLIP
     |████████████████████████████████| 53 kB 1.2 MB/s 
  Running setup.py develop for clip
     |████████████████████████████████| 108 kB 12.7 MB/s 


In [ ]:
# !pip /
# torch==1.7.1 \
# tqdm==4.42.1 \
# numpy==1.18.1 \
# requests==2.22.0 \
# pandas==1.0.1 \
# nltk==3.4.5 \
# six==1.14.0 \
# scikit_image==0.16.2 \
# opencv_python==3.4.9.33 \
# torchvision==0.8.2 \
# torch==1.7.1 \
# dominate==2.5.1 \
# matplotlib==3.1.3 \
# visdom==0.1.8.9 \
# pytorch_pretrained_bert==0.6.2 \
# lmdb==1.0.0 \
# scipy==1.4.1 \
# ipython==7.21.0 \
# Pillow==8.1.0 \
# python_dateutil==2.8.1 \
# rawpy==0.16.0 \
# tensorboardX==2.1 \
# streamlit \
# git+https://github.com/openai/CLIP.git \

In [26]:
import torch
import os
import pickle
import sys
# sys.path.append('./CLIP')
sys.path.append('./stylegan3')
import torch_utils.persistence

with open('stylegan3-t-ffhq-1024x1024.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1], no truncation

In [ ]:
z = torch.randn([1, G.z_dim]).cuda() 
img = G(z, c) 
print(img.shape)
# print(G.z_dim)
# print(z)
import torchvision.transforms.functional as TF
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
tf = Compose([
  Resize(224),
  lambda x: torch.clamp((x+1)/2,min=0,max=1),
  ])
display(TF.to_pil_image(tf(img)[0]))


In [ ]:
# !python /content/stylegan3/no.py

In [88]:
from numpy.core.fromnumeric import resize
from torch.nn.modules.conv import Conv2d
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))
    ])

transform = transforms.ToTensor()

import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

class CDataset(Dataset):
    def __init__(self, transform=None, target_transform=None):
        self.simg_dir = "/content/sketch/"
        self.oimg_dir = "/content/CelebAMask-HQ/CelebA-HQ-img/"
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return 10000

    def __getitem__(self, idx):
        img_path = os.path.join(self.simg_dir, str(idx+100)+".jpg")
        simage = read_image(img_path)
        img_path = os.path.join(self.oimg_dir, str(idx+100)+".jpg")
        oimage = read_image(img_path)
        if self.transform:
            simage = self.transform(simage)
            oimage = self.transform(oimage)
        return simage, oimage

transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize(512),
                                transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])


dataset = CDataset(transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)
# mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# data_loader = torch.utils.data.DataLoader(dataset=mnist_data,
#                                           batch_size=64,
#                                           shuffle=True)
# dataiter = iter(dataloader)
# simg, oimg = dataiter.next()
# print(torch.min(images), torch.max(images))


class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 3, 512, 512
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1), # -> N, 16, 14, 14
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # -> N, 32, 7, 7
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # -> N, 16, 14, 14
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), # -> N, 32, 7, 7
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1), # -> N, 16, 14, 14
            nn.ReLU(),
            nn.Conv2d(256, 512, 3, stride=2, padding=1), # -> N, 16, 14, 14
            nn.ReLU(),
            # nn.Conv2d(512, 1024, 3, stride=2, padding=1), # -> N, 16, 14, 14
            # nn.ReLU(),
            # nn.Conv2d(1024, 1563, 3, stride=2, padding=1), # -> N, 16, 14, 14
            # nn.ReLU(),
            nn.Conv2d(512, 1, 1), # -> N, 64, 1, 1
            nn.MaxPool2d(3)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return encoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

model = Autoencoder()

# criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)
dataiter = iter(dataloader)
s, o = dataiter.next()
ott = model(s)
ot = torch.reshape(ott, (1,512))
img = G(ot, None)
tf = Compose([
  Resize(224),
  lambda x: torch.clamp((x+1)/2,min=0,max=1),
  ])
display(TF.to_pil_image(tf(img)[0]))
# Point to training loop video
# num_epochs = 1
# outputs = []
# for epoch in range(num_epochs):
#     for (img, _) in data_loader:
#         # img = img.reshape(-1, 28*28) # -> use for Autoencoder_Linear
#         recon = model(img)
#         loss = criterion(recon, img)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
#     outputs.append((epoch, img, recon))

RuntimeError: ignored

In [ ]:
torch.reshape(-1, (1,512))

import sys
sys.path.append('./CLIP')
sys.path.append('./stylegan3')

import io
import os, time
import pickle
import shutil
import numpy as np
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import clip
import copy
import imageio
import unicodedata
import re
from PIL import Image
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from IPython.display import display
from einops import rearrange
from google.colab import files
from time import perf_counter
from stylegan3.dnnlib.util import open_url


In [ ]:
import sys
sys.path.append('./CLIP')
sys.path.append('./stylegan3')

import io
import os, time
import pickle
import shutil
import numpy as np
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import clip
import copy
import imageio
import unicodedata
import re
from PIL import Image
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from IPython.display import display
from einops import rearrange
from google.colab import files
from time import perf_counter
from stylegan3.dnnlib.util import open_url


device = torch.device('cuda:0')

# Load VGG16 feature detector.
url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metrics/vgg16.pt'
with open_url(url) as f:
    vgg16 = torch.jit.load(f).eval().to(device)
print('Using device:', device, file=sys.stderr)

from google.colab import drive
drive.mount('/content/drive')

def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

def norm1(prompt):
    "Normalize to the unit sphere."
    return prompt / prompt.square().sum(dim=-1,keepdim=True).sqrt()

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)

class MakeCutouts(torch.nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
        return torch.cat(cutouts)

make_cutouts = MakeCutouts(224, 32, 0.5)

def embed_image(image):
  n = image.shape[0]
  cutouts = make_cutouts(image)
  embeds = clip_model.embed_cutout(cutouts)
  embeds = rearrange(embeds, '(cc n) c -> cc n c', n=n)
  return embeds

# def embed_url(url):
#   image = Image.open(fetch(url)).convert('RGB')
#   return embed_image(TF.to_tensor(image).to(device).unsqueeze(0)).mean(0).squeeze(0)

class CLIP(object):
  def __init__(self):
    clip_model = "ViT-B/32"
    self.model, _ = clip.load(clip_model)
    self.model = self.model.requires_grad_(False)
    self.normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                          std=[0.26862954, 0.26130258, 0.27577711])

  @torch.no_grad()
  def embed_text(self, prompt):
      "Normalized clip text embedding."
      return norm1(self.model.encode_text(clip.tokenize(prompt).to(device)).float())

  def embed_cutout(self, image):
      "Normalized clip image embedding."
      return norm1(self.model.encode_image(self.normalize(image)))
  
clip_model = CLIP()

# Projector

def project(
    G,
    target: torch.Tensor, # [C,H,W] and dynamic range [0,255], W & H must match G output resolution
    *,
    num_steps                  = 1000,
    w_avg_samples              = -1,
    initial_learning_rate      = 0.1,
    initial_noise_factor       = 0.05,
    lr_rampdown_length         = 0.25,
    lr_rampup_length           = 0.05,
    noise_ramp_length          = 0.75,
    regularize_noise_weight    = 1e5,
    verbose                    = False,
    device: torch.device
):

    assert target.shape == (G.img_channels, G.img_resolution, G.img_resolution)

    def logprint(*args):
        if verbose:
            print(*args)

    G = copy.deepcopy(G).eval().requires_grad_(False).to(device) # type: ignore

    # Compute w stats.
    if w_avg_samples > 0:
      logprint(f'Computing W midpoint and stddev using {w_avg_samples} samples...')
      z_samples = np.random.RandomState(123).randn(w_avg_samples, G.z_dim)
    else:
      seed = np.random.randint(0, 2**32 - 1)
      z_samples = np.random.RandomState(seed).randn(1, G.z_dim)
    w_samples = G.mapping(torch.from_numpy(z_samples).to(device), None)  # [N, L, C]
    w_samples = w_samples[:, :1, :].cpu().numpy().astype(np.float32)       # [N, 1, C]
    w_avg = np.mean(w_samples, axis=0, keepdims=True)      # [1, 1, C]
    w_std = (np.sum((w_samples - w_avg) ** 2) / w_avg_samples) ** 0.5

    # Setup noise inputs.
    noise_bufs = { name: buf for (name, buf) in G.synthesis.named_buffers() if 'noise_const' in name }

    # Features for target image.
    target_images = target.unsqueeze(0).to(device).to(torch.float32)
    if target_images.shape[2] > 256:
        target_images = F.interpolate(target_images, size=(256, 256), mode='area')
    target_features = vgg16(target_images, resize_images=False, return_lpips=True)

    w_opt = torch.tensor(w_avg, dtype=torch.float32, device=device, requires_grad=True) # pylint: disable=not-callable
    w_out = torch.zeros([num_steps] + list(w_opt.shape[1:]), dtype=torch.float32, device=device)
    optimizer = torch.optim.Adam([w_opt] + list(noise_bufs.values()), betas=(0.9, 0.999), lr=initial_learning_rate)

    # Init noise.
    for buf in noise_bufs.values():
        buf[:] = torch.randn_like(buf)
        buf.requires_grad = True

    for step in range(num_steps):
        # Learning rate schedule.
        t = step / num_steps
        w_noise_scale = w_std * initial_noise_factor * max(0.0, 1.0 - t / noise_ramp_length) ** 2
        lr_ramp = min(1.0, (1.0 - t) / lr_rampdown_length)
        lr_ramp = 0.5 - 0.5 * np.cos(lr_ramp * np.pi)
        lr_ramp = lr_ramp * min(1.0, t / lr_rampup_length)
        lr = initial_learning_rate * lr_ramp
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # Synth images from opt_w.
        w_noise = torch.randn_like(w_opt) * w_noise_scale
        ws = (w_opt + w_noise).repeat([1, G.mapping.num_ws, 1])
        synth_images = G.synthesis(ws, noise_mode='const')

        # Downsample image to 256x256 if it's larger than that. VGG was built for 224x224 images.
        synth_images = (synth_images + 1) * (255/2)
        if synth_images.shape[2] > 256:
            synth_images = F.interpolate(synth_images, size=(256, 256), mode='area')

        # Features for synth images.
        synth_features = vgg16(synth_images, resize_images=False, return_lpips=True)
        dist = (target_features - synth_features).square().sum()

        # Noise regularization.
        reg_loss = 0.0
        for v in noise_bufs.values():
            noise = v[None,None,:,:] # must be [1,1,H,W] for F.avg_pool2d()
            while True:
                reg_loss += (noise*torch.roll(noise, shifts=1, dims=3)).mean()**2
                reg_loss += (noise*torch.roll(noise, shifts=1, dims=2)).mean()**2
                if noise.shape[2] <= 8:
                    break
                noise = F.avg_pool2d(noise, kernel_size=2)
        loss = dist + reg_loss * regularize_noise_weight

        # Step
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        logprint(f'step {step+1:>4d}/{num_steps}: dist {dist:<4.2f} loss {float(loss):<5.2f}')

        # Save projected W for each optimization step.
        w_out[step] = w_opt.detach()[0]

        # Normalize noise.
        with torch.no_grad():
            for buf in noise_bufs.values():
                buf -= buf.mean()
                buf *= buf.square().mean().rsqrt()

    return w_out.repeat([1, G.mapping.num_ws, 1])

def get_perceptual_loss(synth_image, target_features):
    # Downsample image to 256x256 if it's larger than that. VGG was built for 224x224 images.
    synth_image = (synth_image + 1) * (255/2)
    if synth_image.shape[2] > 256:
        synth_image = F.interpolate(synth_image, size=(256, 256), mode='area')

    # Features for synth images.
    synth_features = vgg16(synth_image, resize_images=False, return_lpips=True)
    return (target_features - synth_features).square().sum()

def get_target_features(target):
    target_images = target.unsqueeze(0).to(device).to(torch.float32)
    if target_images.shape[2] > 256:
        target_images = F.interpolate(target_images, size=(256, 256), mode='area')
    return vgg16(target_images, resize_images=False, return_lpips=True)

#@markdown #**Model selection** 🎭


#@markdown There are 4 pre-trained options to play with:
#@markdown - FFHQ: Trained with human faces.
#@markdown - MetFaces: Trained with paintings/portraits of human faces.
#@markdown - AFHQv2: Trained with animal faces.
#@markdown - Cosplay: Trained by [l4rz](https://twitter.com/l4rz) with cosplayer's faces.
#@markdown - Wikiart: Trained by [Justin Pinkney](https://www.justinpinkney.com/) with the Wikiart 1024 dataset.
#@markdown - Landscapes: Trained by [Justin Pinkney](https://www.justinpinkney.com/) with the LHQ dataset.


#@markdown **Run this cell again if you change the model**.

#@markdown ---

base_url = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/"

Model = 'FFHQ' #@param ["FFHQ", "MetFaces", "AFHQv2", "cosplay", "Wikiart", "Landscapes"]

#@markdown ---

model_name = {
    "FFHQ": base_url + "stylegan3-t-ffhqu-1024x1024.pkl",
    "MetFaces": base_url + "stylegan3-r-metfacesu-1024x1024.pkl",
    "AFHQv2": base_url + "stylegan3-t-afhqv2-512x512.pkl",
    "cosplay": "https://l4rz.net/cosplayface-snapshot-stylegan3t-008000.pkl",
    "Wikiart": "https://drive.google.com/u/0/open?id=18MOpwTMJsl_Z17q-wQVnaRLCUFZYSNkj",
    "Landscapes": "https://drive.google.com/u/0/open?id=14UGDDOusZ9TMb-pOrF0PAjMGVWLSAii1"
}

network_url = model_name[Model]

with open(fetch_model(network_url), 'rb') as fp:
  G = pickle.load(fp)['G_ema'].to(device)

zs = torch.randn([10000, G.mapping.z_dim], device=device)
w_stds = G.mapping(zs, None).std(0)

#@markdown #**Parameters** ✍️
#@markdown ---

target_image_filename = ""#@param {type:"string"}
text = ""#@param {type:"string"}
loss_ratio = 0.4#@param {type:"number"}
steps = 800#@param {type:"number"}
limit_step = 600#@param {type:"number"}
seed = 14#@param {type:"number"}

#@markdown Choose -1 for a random seed.

#@markdown ---

if seed == -1:
    seed = np.random.randint(0,9e9)

target = clip_model.embed_text(text)

target_pil = Image.open(target_image_filename).convert('RGB')
w, h = target_pil.size
s = min(w, h)
target_pil = target_pil.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
target_pil = target_pil.resize((G.img_resolution, G.img_resolution), Image.LANCZOS)
target_uint8 = np.array(target_pil, dtype=np.uint8)
target_tensor = torch.tensor(target_uint8.transpose([2, 0, 1]), device=device)

#@markdown #**Run the model** 🚀

# Actually do the run

tf = Compose([
  Resize(224),
  lambda x: torch.clamp((x+1)/2,min=0,max=1),
])


def run(timestring, projection_target):
  torch.manual_seed(seed)

  target_features = get_target_features(projection_target)

  # Init
  # Sample 32 inits and choose the one closest to prompt

  with torch.no_grad():
    qs = []
    losses = []
    for _ in range(8):
      q = (G.mapping(torch.randn([4,G.mapping.z_dim], device=device), None, truncation_psi=0.7) - G.mapping.w_avg) / w_stds
      images = G.synthesis(q * w_stds + G.mapping.w_avg)
      loss = get_perceptual_loss(images, target_features)
      i = torch.argmin(loss)
      qs.append(q[i])
      losses.append(loss)
    qs = torch.stack(qs)
    losses = torch.stack(losses)
    i = torch.argmin(losses)
    q = qs[i].unsqueeze(0).requires_grad_()

  # Sampling loop
  q_ema = q
  opt = torch.optim.AdamW([q], lr=0.03, betas=(0.0,0.999))
  loop = tqdm(range(steps))
  for i in loop:
    opt.zero_grad()
    w = q * w_stds
    image = G.synthesis(w + G.mapping.w_avg, noise_mode='const')
    embed = embed_image(image.add(1).div(2))
    step_ratio = i / limit_step
    perceptual_loss = get_perceptual_loss(image, target_features)
    modulated_perceptual_loss = (
        max(loss_ratio, 1 - step_ratio)
        * get_perceptual_loss(image, target_features)
    )
    clip_loss = spherical_dist_loss(embed, target).mean()
    modulated_clip_loss = (
        min(1 - loss_ratio, step_ratio)
        * (step_ratio) * spherical_dist_loss(embed, target).mean()
    )
    loss = modulated_perceptual_loss + modulated_clip_loss
    loss.backward()
    opt.step()
    loop.set_postfix(loss=loss.item(), q_magnitude=q.std().item())

    q_ema = q_ema * 0.9 + q * 0.1
    image = G.synthesis(q_ema * w_stds + G.mapping.w_avg, noise_mode='const')

    if i % 10 == 0:
      display(TF.to_pil_image(tf(image)[0]))
      print(f"image {i}/{steps} | projector loss: {perceptual_loss} | clip loss: {clip_loss} | modulated loss: {loss}")
    pil_image = TF.to_pil_image(image[0].add(1).div(2).clamp(0,1))
    os.makedirs(f'samples/{timestring}', exist_ok=True)
    pil_image.save(f'samples/{timestring}/{i:04}.jpg')

try:
  timestring = time.strftime('%Y%m%d%H%M%S')
  run(timestring, target_tensor)
except KeyboardInterrupt:
  pass

#@markdown #**Save images** 📷
#@markdown A `.tar` file will be saved inside *samples* and automatically downloaded, unless you previously ran the Google Drive cell,
#@markdown in which case it'll be saved inside your previously created drive *samples* folder.

archive_name = "optional"#@param {type:"string"}

archive_name = slugify(archive_name)

if archive_name != "optional":
  fname = archive_name
  # os.rename(f'samples/{timestring}', f'samples/{fname}')
else:
  fname = timestring
# Save images as a tar archive
!tar cf samples/{fname}.tar samples/{timestring}
if os.path.isdir('drive/MyDrive/samples'):
  shutil.copyfile(f'samples/{fname}.tar', f'drive/MyDrive/samples/{fname}.tar')
else:
  files.download(f'samples/{fname}.tar')

#@markdown #**Generate video** 🎥

#@markdown You can edit frame rate and stuff by double-clicking this tab.

frames = os.listdir(f"samples/{timestring}")
frames = len(list(filter(lambda filename: filename.endswith(".jpg"), frames))) #Get number of jpg generated

init_frame = 1 #This is the frame where the video will start
last_frame = frames #You can change i to the number of the last frame you want to generate. It will raise an error if that number of frames does not exist.

min_fps = 10
max_fps = 60

total_frames = last_frame-init_frame

#Desired video time in seconds
video_length = 14 #@param {type:"number"}
#Video filename
video_name = "" #@param {type:"string"}
video_name = slugify(video_name)

# frames = []
# tqdm.write('Generating video...')
# for i in range(init_frame,last_frame): #
#     filename = f"samples/{timestring}/{i:04}.jpg"
#     frames.append(Image.open(filename))

fps = np.clip(total_frames/video_length,min_fps,max_fps)

!ffmpeg -r {fps} -i samples/{timestring}/%04d.jpg -c:v libx264 -vf fps={fps} -pix_fmt yuv420p samples/{video_name}.mp4 -frames:v {total_frames}

# from subprocess import Popen, PIPE
# p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', f'samples/{video_name}.mp4'], stdin=PIPE)
# for im in tqdm(frames):
#     im.save(p.stdin, 'PNG')
# p.stdin.close()

print("The video is now being compressed, wait...")
p.wait()
print("The video is ready")

#@markdown #**Download video** 📀
#@markdown If you're activated the download to GDrive option, the video will be save there. Don't worry about overwritting issues for colliding filenames, an id will be added to them to avoid this.

#Video filename
to_download_video_name = "" #@param {type:"string"}
to_download_video_name = slugify(to_download_video_name)

from google.colab import files
if os.path.isdir('drive/MyDrive/samples'):
  filelist = glob.glob(f'drive/MyDrive/samples/{to_download_video_name}*.mp4')
  video_count = len(filelist)
  if video_count:
    final_video_name = f"{to_download_video_name}{video_count}"
  else:
    final_video_name = to_download_video_name
  shutil.copyfile(f'samples/{video_name}.mp4', f'drive/MyDrive/samples/{final_video_name}.mp4')
else:
  files.download(f"{to_download_video_name}.mp4")